In [1]:
"""
자연어 처리(NLP) 스팸, 뉴스/쇼핑 카테고리 분류, 텍스트 요약, 문장 생성, 기계 번역,
챗봇 등 다양한 분야에 활용되고 있다

-자연어 처리 방법
텍스트 데이터는 테이블 데이터와 같이 구조화되거나 데이터의 길이가 일정하지 않다는
특성을 갖는다
따라서 문장의 길이가 다를 경우 딥러닝 모델에 입력으로 넣기 위해서는 길이를 동일하게
맞추는 작업이 선행되어야 한다 
가장 긴 문장의 길이에 맞출 수도 있고, 가장 짧은 문장 길이에 맞춰 넘치는 부분을 
잘라 낼 수도 있다

또한 한글을 입력 데이터로 활용하기 위해서는 숫자로 변환해야 하고, 한글과 숫자를 일대일
매칭하는 방식으로 단어 사전을 만든다 
여기서 말하는 사전은 "텐서플로" : "2021", "딥러닝" : "2022" 와 같이 
모든 단어 (토큰)를 숫자로 매핑한 사전이다
사전을 통해 단어를 숫자로, 숫자를 단어로 변경할 수 있다

이렇게 단어(토큰)를 숫자로 변경하기 위해서는 문장을 특정한 기준으로 잘라내야 하는데,
이렇게 잘라낸 조각을 토큰이라고 표현한다 
영어에서는 띄어쓰기를 기준으로 잘라내더라도 큰 문제가 없으나 
한글에서는 띄어쓰기가 잘 되어 있지 않고 '~이(가)', '~을(를)', '~에게' 등
조사(명사에 붙어서 다른 말과의 관계를 나타내거나 특별한 뜻을 더해 주는 품사)가 붙어 있어
잘라내는 것에 어려움이 있다

예를 들어 "텐서플로가", "텐서플로를", "텐서플로는"이 있다면, 컴퓨터는 3가지 모두 다른
단어로 인식한다. 이에 띄어쓰기는 되어있지 않지만 "텐서플로"를 구분하는 토크나이저를 
활용하거나 불용어(stopword) 처리를 통해 "가", "를", "는" 등 조사나 반복되는 불필요한 
단어를 제거해야한다 

다음 예제에서는 문장 텍스트 데이터를 처리해 딥러닝 모델에 입력으로 넣기 직전의 
데이터셋을 준비하는 방법을 알아본다
- 토큰화 ( 문장을 띄어쓰기 기준으로 나눔) + 단어 사전(단어와 숫자 매칭)
- 문자 인코딩(사전을 바탕으로 문장들을 숫자로 변경)
- 인코딩된 문장 길이를 동일하게 변경(패딩)

"""

"""
텐서플로에서 제공하는 Tokenizer는 띄어쓰기를 기준으로 단어 인코딩 사전을 생성하고,
단어를 쉽게 인코딩할 수 있게 도와준다 
앞에서 그림으로 설명한 문장을 실제 케라스를 활용해서 단어 사전으로 변환한다
Tokenizer 객체를 생성하고, fit_on_texts() 함수에 인코딩할 문장들을 입력한 뒤 
단어 토큰을 만들고 각각 인덱스를 지정한다 
"""
#텐서플로 토크나이저
from tensorflow.keras.preprocessing.text import Tokenizer
sentences = [ "영실이는 나를 정말 정말 좋아해", 
            "영실이는 영화를 좋아해"]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
print("단어 인덱스", tokenizer.word_index)

단어 인덱스 {'영실이는': 1, '정말': 2, '좋아해': 3, '나를': 4, '영화를': 5}


In [2]:
"""
texts_to_sequences() 함수는 입력된 문장을 단어 인덱스를 사용하여 숫자 벡터로 변환
"""
# 인코딩된 결과
word_encoding = tokenizer.texts_to_sequences(sentences)
word_encoding

[[1, 4, 2, 2, 3], [1, 5, 3]]

In [3]:
"""
만약 사전에 없는 새로운 단어가 등장하면 새로운 단어는 인터딩할 때 무시한다
다음 코드를 보면 "경록이와" 라는 단어는 앞에서 만든 사전에 없으므로 인코딩된 
숫자에서 빠져있음을 확인할 수 있다
"""
# 사전에 없는 단어가 있을 때 인코딩 결과
new_sentences = ["영실이는 경록이와 나를 좋아해"]
new_word_encoding = tokenizer.texts_to_sequences(new_sentences)
new_word_encoding

[[1, 4, 3]]

In [4]:
"""
앞의 코드와 같이,  자연어에는 사전에 없는 새로운 단어가 등장할 수 있다
사전에 존재하지 않는 단어를 OOV(Out of Vocabulary) token이라고 부른다 
케라스 Tokenizer는 이를 처리하기 위해 oov_token 파라미터 값을 설정할 수 있다
"""
# 사전에 없는 (Out of Vocabulary) 단어 처리 - oov : 경록이와 
tokenizer = Tokenizer(oov_token = "<OOV>")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

new_word_encoding = tokenizer.texts_to_sequences(new_sentences)

print(word_index)
print(new_word_encoding)

{'<OOV>': 1, '영실이는': 2, '정말': 3, '좋아해': 4, '나를': 5, '영화를': 6}
[[2, 1, 5, 4]]


In [5]:
"""
텍스트 데이터셋에 빈도수가 작은 단어가 많이 존재하는 경우에는 이들 단어를 제외하는
것이 일반적이다 
즉 문장을 토큰으로 인코딩할 때 빈도수가 많은 순서대로 최대 사전 개수를 정하고
빈도수가 적은 단어를 제외한다
최대 사전 개수는 num_words 파라미터를 통해 설정한다
3으로 설정하는 경우 빈도수순으로 3개 토큰만 인코딩하고, 나머지 단어 "1"(OOV)로 
인코딩될 것이다
"""
# 단어 사전 개수 설정
tokenizer = Tokenizer(num_words=3, oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

new_word_encoding = tokenizer.texts_to_sequences(new_sentences)

print(word_index)
print(new_word_encoding)

{'<OOV>': 1, '영실이는': 2, '정말': 3, '좋아해': 4, '나를': 5, '영화를': 6}
[[2, 1, 1, 1]]


In [6]:
"""
순환신경망(RNN)에 데이터를 입력으로 넣기 위해서는 문장의 길이(크기)를 동일하게 
맞춰야 한다 
이를 패딩(padding)이라고 부른다

케라스에서 제공하는 pad_sequences() 함수를 활용하면 인코딩된 문자의 길이를 동일하게
만들수 있다. 최대 문장의 길이를 기준으로 그보다 짧다면 앞에 0 값이 채워지는 것을 
확인할 수 있다

"""
# 문장의 길이 맞추기
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded = pad_sequences(word_encoding)
print(padded)

[[1 4 2 2 3]
 [0 0 1 5 3]]


In [7]:
"""
한편, 뒤쪽에 0을 채우기 위해서는 padding 파라미터 값을 "post"로 설정한다
값을 출력해보면 뒤쪽으로 부족한 개수만큼 0으로 채워지는 것을 확인할 수 있다
"""
#패딩(뒤에 0 붙이기)
padded = pad_sequences(word_encoding, padding="post")
print(padded)

[[1 4 2 2 3]
 [1 5 3 0 0]]


In [8]:
"""
만약 몇몇 문장만 길이가 길고 대부분의 문장 길이(단어 개수)가 4 이하라면 최대값을 
4로 설정 할 수도 있다
"""
# 문장의 최대 길이 고정
padded = pad_sequences(word_encoding, padding="post", maxlen=4)
print(padded)

[[4 2 2 3]
 [1 5 3 0]]


In [9]:
"""
이때 최대 길이인 4보다 긴 문장의 경우 잘라내야 하며, padding 과 동일하게 기본값은 
앞부분이 잘리게 된다, 뒷부분을 자르고 싶을 때는 truncating 파라미터 값을 "post"로 
설정하면 된다
"""
# 최대 길이보다 문장이 길 때 뒷부분 자르기
padded = pad_sequences(word_encoding, padding="post", truncating="post", maxlen=4)
print(padded)

[[1 4 2 2]
 [1 5 3 0]]
